In [1]:
from collections import defaultdict, Counter
import gzip,  math

In [39]:
akl = dict( [ (x+'-n', True) for x in 'focus, ability, absence, account, achievement, act, action, activity, addition, adoption, adult, advance, advantage, advice, age, aim, alternative, amount, analogy, analysis, application, approach, argument, aspect, assertion, assessment, assistance, association, assumption, attempt, attention, attitude, author, awareness, balance, basis, behaviour, behavior, being, belief, benefit, bias, birth, capacity, case, category, cause, centre, challenge, change, character, characteristic, choice, circumstance, class, classification, code, colleague, combination, commitment, committee, communication, community, comparison, complexity, compromise, concentration, concept, conception, concern, conclusion, condition, conduct, conflict, consensus, consequence, consideration, constraint, construction, content, contradiction, contrast, contribution, control, convention, correlation, country, creation, crisis, criterion, criticism, culture, damage, data, debate, decision, decline, defence, defense, definition, degree, demand, description, destruction, determination, development, difference, difficulty, dilemma, dimension, disadvantage, discovery, discrimination, discussion, distinction, diversity, division, doctrine, effect, effectiveness, element, emphasis, environment, error, essence, establishment, evaluation, event, evidence, evolution, examination, example, exception, exclusion, existence, expansion, experience, experiment, explanation, exposure, extent, extreme, fact, factor, failure, feature, female, figure, finding, force, form, formation, function, future, gain, group, growth, guidance, guideline, hypothesis, idea, identity, impact, implication, importance, improvement, increase, indication, individual, influence, information, insight, instance, institution, integration, interaction, interest, interpretation, intervention, introduction, investigation, isolation, issue, kind, knowledge, lack, learning, level, likelihood, limit, limitation, link, list, literature, logic, loss, maintenance, majority, male, manipulation, mankind, material, means, measure, medium, member, method, minority, mode, model, motivation, movement, need, network, norm, notion, number, observation, observer, occurrence, operation, opportunity, option, organisation, organization, outcome, output, paper, parallel, parent, part, participant, past, pattern, percentage, perception, period, person, personality, perspective, phenomenon, point, policy, population, position, possibility, potential, practice, presence, pressure, problem, procedure, process, production, programme, program, progress, property, proportion, proposition, protection, provision, publication, purpose, quality, question, range, rate, reader, reality, reason, reasoning, recognition, reduction, reference, relation, relationship, relevance, report, representative, reproduction, requirement, research, resistance, resolution, resource, respect, restriction, result, review, rise, risk, role, rule, sample, scale, scheme, scope, search, section, selection, sense, separation, series, service, set, sex, shift, significance, similarity, situation, skill, society, solution, source, space, spread, standard, statistics, stimulus, strategy, stress, structure, subject, success, summary, support, survey, system, target, task, team, technique, tendency, tension, term, theme, theory, tolerance, topic, tradition, transition, trend, type, uncertainty, understanding, unit, use, validity, value, variation, variety, version, view, viewpoint, volume, whole, work, world'.split(', ') ]+\
            [ (x+'-v', True)  for x in 'accept, account, achieve, acquire, act, adapt, adopt, advance, advocate, affect, aid, aim, allocate, allow, alter, analyse, analyze, appear, apply, argue, arise, assert, assess, assign, associate, assist, assume, attain, attempt, attend, attribute, avoid, base, be, become, benefit, can, cause, characterise, characterize, choose, cite, claim, clarify, classify, coincide, combine, compare, compete, comprise, concentrate, concern, conclude, conduct, confine, conform, connect, consider, consist, constitute, construct, contain, contrast, contribute, control, convert, correspond, create, damage, deal, decline, define, demonstrate, depend, derive, describe, design, destroy, determine, develop, differ, differentiate, diminish, direct, discuss, display, distinguish, divide, dominate, effect, eliminate, emerge, emphasize, employ, enable, encounter, encourage, enhance, ensure, establish, evaluate, evolve, examine, exceed, exclude, exemplify, exist, expand, experience, explain, expose, express, extend, facilitate, fail, favour, favor, finance, focus, follow, form, formulate, function, gain, generate, govern, highlight, identify, illustrate, imply, impose, improve, include, incorporate, increase, indicate, induce, influence, initiate, integrate, interpret, introduce, investigate, involve, isolate, label, lack, lead, limit, link, locate, maintain, may, measure, neglect, note, obtain, occur, operate, outline, overcome, participate, perceive, perform, permit, pose, possess, precede, predict, present, preserve, prevent, produce, promote, propose, prove, provide, publish, pursue, quote, receive, record, reduce, refer, reflect, regard, regulate, reinforce, reject, relate, rely, remain, remove, render, replace, report, represent, reproduce, require, resolve, respond, restrict, result, retain, reveal, seek, select, separate, should, show, solve, specify, state, stimulate, strengthen, stress, study, submit, suffer, suggest, summarise, summarize, supply, support, sustain, tackle, tend, term, transform, treat, undermine, undertake, use, vary, view, write, yield'.split(', ') ]+\
            [ (x+'-adj', True) for x in 'absolute, abstract, acceptable, accessible, active, actual, acute, additional, adequate, alternative, apparent, applicable, appropriate, arbitrary, available, average, basic, central, certain, clear, common, competitive, complete, complex, comprehensive, considerable, consistent, conventional, correct, critical, crucial, dependent, detailed, different, difficult, distinct, dominant, early, effective, equal, equivalent, essential, evident, excessive, experimental, explicit, extensive, extreme, far, favourable, favorable, final, fixed, following, formal, frequent, fundamental, future, general, great, high, human, ideal, identical, immediate, important, inadequate, incomplete, independent, indirect, individual, inferior, influential, inherent, initial, interesting, internal, large, late, leading, likely, limited, local, logical, main, major, male, maximum, mental, minimal, minor, misleading, modern, mutual, natural, necessary, negative, new, normal, obvious, original, other, overall, parallel, partial, particular, passive, past, permanent, physical, positive, possible, potential, practical, present, previous, primary, prime, principal, productive, profound, progressive, prominent, psychological, radical, random, rapid, rational, real, realistic, recent, related, relative, relevant, representative, responsible, restricted, scientific, secondary, selective, separate, severe, sexual, significant, similar, simple, single, so-called, social, special, specific, stable, standard, strict, subsequent, substantial, successful, successive, sufficient, suitable, surprising, symbolic, systematic, theoretical, total, traditional, true, typical, unique, unlike, unlikely, unsuccessful, useful, valid, valuable, varied, various, visual, vital, wide, widespread'.split(', ') ]+\
            [ (x+'-adv', True) for x in 'above, accordingly, accurately, adequately, also, approximately, at best, basically, clearly, closely, commonly, consequently, considerably, conversely, correctly, directly, effectively, e.g., either, equally, especially, essentially, explicitly, extremely, fairly, far, for example, for instance, frequently, fully, further, generally, greatly, hence, highly, however, increasingly, indeed, independently, indirectly, inevitably, initially, in general, in particular, largely, less, mainly, more, moreover, most, namely, necessarily, normally, notably, often, only, originally, over, partially, particularly, potentially, previously, primarily, purely, readily, recently, relatively, secondly, significantly, similarly, simply, socially, solely somewhat, specifically, strongly, subsequently, successfully, thereby, therefore, thus, traditionally, typically, ultimately, virtually, wholly, widely'.split(', ') ] )

In [40]:
test = [ x for x in akl.keys() ][:7]

In [2]:
# d 跟 head tail
skipbigram_d = defaultdict(lambda: defaultdict(lambda: Counter()) )
# key col d example
skipbigram_ex = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: [])) )

In [3]:
for line in gzip.open('citeseerx.ngms.gz', 'rt'):
    pattern, count = line.split('\t')
    tokens, count = pattern.split(' '), int(count)
    d_len = len(tokens)
    head, tail = tokens[0], tokens[-1]
    
    skipbigram_d[head][tail][d_len-1] += count
    skipbigram_d[tail][head][1-d_len ] += count
    skipbigram_ex[head][tail][d_len-1].append((pattern, count))
    skipbigram_ex[tail][head][1-d_len].append((pattern, count))

In [5]:
#skipbigram_ex['focus-n']['on-prep']

In [6]:
# 統計量
import numpy as np
skipbigram_static = defaultdict(lambda: defaultdict())

In [41]:
# 計算塞選條件
for head in skipbigram_d:
    for col in skipbigram_d[head]:
        values = list(skipbigram_d[head][col].values())
        #計算統計量
        freq, avg_f = np.sum(values), np.mean(values)
        #average positional co-occurring frequency
        avg_p = freq/10
        #spread of all pij
        spread = math.sqrt(sum([ (skipbigram_d[head][col][j] - avg_p)**2/10 for j in range(-5, 6)]))
        skipbigram_static[head][col] = {'freq': freq, 'avg_f': avg_f, 
                                     'avg_p': avg_p, 'spread': spread}

In [8]:
#skipbigram_d['focus-n']['be-v']
#skipbigram_ex['focus-n']['be-v']

Counter({-3: 919, -2: 2916, -1: 102, 1: 7248, 2: 1738, 3: 95, 4: 4151, 5: 150})

In [43]:
#skipbigram_static['focus-n']

In [52]:
for key in ['advice-n']:
    N = len(skipbigram_static[key])
    averge_f = sum([ static['freq'] for col, static in skipbigram_static[key].items() ]) / N 
    std = math.sqrt(sum([ (static['freq'] - static['avg_f'])**2 / N for col, static in skipbigram_static[key].items() ]))
    #Condition1 strength  filter出現頻率不高的col
    filter_c = filter(lambda x: (skipbigram_static[key][x[0]]['freq'] - avg_f) / std > 1, skipbigram_d[key].items())
    
    #Condition2 衡量周邊詞在目標詞 ±d 跨距內各個位置的分佈情形，若周邊詞在各個位置分佈過於分散且次數平均，則將其過濾掉
    filter_c = filter(lambda x: skipbigram_static[key][x[0]]['spread'] > 10, filter_c)
    
    print('\n')
    print(key)
    for coll, b_d in filter_c:
        #Condition3 find最適當的距離
        b_d = max(filter(lambda x: x[1] > skipbigram_static[key][coll]['avg_p'] + math.sqrt(skipbigram_static[key][coll]['spread']), b_d.items()), key=lambda x: x[1])
        print(coll, skipbigram_static[key][coll]['freq'] ,'\t', max(skipbigram_ex[key][coll][b_d[0]], key=lambda x: x[1]))




advice-n
be-v 622 	 ('advice-n be-v', 432)
for-prep 1490 	 ('for-prep his-pron advice-n', 230)
from-prep 457 	 ('advice-n from-prep', 429)
give-v 433 	 ('advice-n give-v', 207)
in-prep 344 	 ('advice-n in-prep', 313)
on-prep 1044 	 ('advice-n on-prep', 950)
provide-v 560 	 ('provide-v advice-n', 370)
with-prep 370 	 ('with-prep advice-n', 116)
valuable-adj 261 	 ('valuable-adj advice-n', 261)
guidance-n 325 	 ('guidance-n and-c advice-n', 113)
support-n 412 	 ('support-n and-c advice-n', 159)
